In [1]:
pwd

'/Users/samuel.weller/Python/gone-phishin-xgov-hackathon/notebooks'

In [2]:
cd ..

/Users/samuel.weller/Python/gone-phishin-xgov-hackathon


In [3]:
from constants import GOV_API_KEY, PST_PHONE_NUMBER, IPHONE_PHONE_NUMBER

In [4]:
from flask_app.app import score

from notifications_python_client.notifications import NotificationsAPIClient

notifications_client = NotificationsAPIClient(GOV_API_KEY)

<regex.Match object; span=(30, 61), match='https://nevincan.av.tr/otp.html'>
['https://nevincan.av.tr/otp.html']


In [5]:
import flask
from flask import Flask, request
import json
import pandas as pd
import pickle
import xgboost
from flask_app.extraction import url_extractor

import regex as re

import urllib.parse as urlparse
from tld import get_tld
import urllib.request as urlreq

In [6]:
### Greeting msg / autoreply msg
greeting_msg_template_id = 'fbd7abfa-1bf3-4fc2-9948-4a37a6ce9293'

#### Template Spam Msgs
spam_msg_tempalte_id = "ed5edbf5-76e0-42be-9c06-3b3717ce00ad"
not_spam_msg_template_id = "5e107ed8-c631-492f-ad7d-91c73919a42e"

In [7]:
### Send text message:
# phone_number = PST_PHONE_NUMBER

# response = notifications_client.send_sms_notification(
#     phone_number=phone_number, # required string
#     template_id=greeting_msg_template_id, # required UUID string
# )

In [8]:
response

NameError: name 'response' is not defined

In [9]:
def get_received_texts():
    """Get all the recieved text messages to the digital phone number"""
    return notifications_client.get_received_texts()

In [10]:
def get_spam_score(url):
    
    """Get the spam score of a given url"""
    
    # Convert to features df
    data = url_extractor(url)
    
    # Load model
    model = pickle.load(open('xg_model.pkl', 'rb'))
    
    spam_score = round(model.predict_proba(data.to_numpy())[0][1], 2)

    return spam_score

In [11]:
def get_spam_classification(spam_score:float, threshold:float=0.75):
    """Get the text classfication or a spam score"""
    if spam_score >= threshold:
        spam_classfication = "SPAM"
    else:
        spam_classfication = "NOT SPAM"
    
    return spam_classfication

In [12]:
google_url = "https://google.com"

google_spam_score = get_spam_score(google_url)

google_spam_score

<regex.Match object; span=(0, 18), match='https://google.com'>
['https://google.com']


0.28

In [13]:
get_spam_classification(spam_score=google_spam_score)

'NOT SPAM'

In [14]:
spam_url = "https://evri.id-60.com"

spam_eg_spam_score = get_spam_score(spam_url)

spam_eg_spam_score

<regex.Match object; span=(0, 22), match='https://evri.id-60.com'>
['https://evri.id-60.com']


0.85

In [15]:
get_spam_classification(spam_score=spam_eg_spam_score)

'SPAM'

In [26]:
import time
texts = get_received_texts()

existing = []

for item in texts['received_text_messages']:
    existing.append(item['id'])
    print(item)

print(existing)

while True:

    # Get texts
    texts = get_received_texts()

    for item in texts['received_text_messages']:

        if item['id'] in existing:
            break

        print(item)
        
        try:
            # Get score and return text
            print("Getting: " + item['content'].lower())
            print(get_spam_score(item['content'].lower()))
            if get_spam_score(item['content'].lower()) > 0.75:
                
                # Return SPAM message
                response = notifications_client.send_sms_notification(
                    phone_number=item['user_number'], # required string
                    template_id=spam_msg_tempalte_id, # required UUID string
                )

            else:

                # Return NOT SPAM
                response = notifications_client.send_sms_notification(
                    phone_number=item['user_number'], # required string
                    template_id=not_spam_msg_template_id, # required UUID string
                )

            #Add to existing
            existing.append(item['id'])
        except Exception as e:
            print("some sort of error occurred, let's hope it's not important")
            print(e)

    time.sleep(1)

{'content': 'Hello I have this dodgy link - www.LinkedIn.com', 'created_at': '2023-03-31T09:18:50.443698Z', 'id': '232904b5-5772-4766-8144-3bcd87c07a37', 'notify_number': '07860003294', 'service_id': '6bcf111f-3cfc-4c63-87fa-5ed3b2c9d76a', 'user_number': '447935993222'}
{'content': 'www.google.co.uk', 'created_at': '2023-03-31T08:45:11.786192Z', 'id': '0833fc6b-35e2-4661-8c64-040c29e8f7d3', 'notify_number': '07860003294', 'service_id': '6bcf111f-3cfc-4c63-87fa-5ed3b2c9d76a', 'user_number': '447923370598'}
{'content': 'https://evri.id-35.com', 'created_at': '2023-03-31T08:44:58.352047Z', 'id': '235153a5-1e27-4b24-bd1c-6c4f0a311eed', 'notify_number': '07860003294', 'service_id': '6bcf111f-3cfc-4c63-87fa-5ed3b2c9d76a', 'user_number': '447923370598'}
{'content': 'https://evri.id-35.com', 'created_at': '2023-03-31T08:44:16.522418Z', 'id': '1aa336a3-88dc-420f-8b55-450f824f2b73', 'notify_number': '07860003294', 'service_id': '6bcf111f-3cfc-4c63-87fa-5ed3b2c9d76a', 'user_number': '447923370598

KeyboardInterrupt: 

In [4]:
from flask_app.extraction import url_extractor

print(url_extractor('www.google.com/'))

<regex.Match object; span=(0, 14), match='www.google.com'>
['http://www.google.com']
   qty_dot_url  qty_hyphen_url  qty_underline_url  qty_slash_url  \
0            2               0                  0              2   

   qty_questionmark_url  qty_equal_url  qty_at_url  qty_and_url  \
0                     0              0           0            0   

   qty_asterisk_url  qty_tld_url  ...  qty_slash_params  \
0                 0            0  ...                 0   

   qty_questionmark_params  qty_equal_params  qty_at_params  qty_and_params  \
0                        0                 0              0               0   

   qty_percent_params  params_length  tld_present_params  qty_params  \
0                   0              0                   0           1   

   email_in_url  
0             0  

[1 rows x 38 columns]
